In [1]:
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-2.2.5/en_core_web_lg-2.2.5.tar.gz

     |████████████████████████████████| 827.9 MB 3.2 kB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-py3-none-any.whl size=829180942 sha256=9e4dc3c70439696b3bdfc43bb4746b0441747e0a99349fe27b5040a223f95f80
  Stored in directory: /root/.cache/pip/wheels/11/95/ba/2c36cc368c0bd339b44a791c2c1881a1fb714b78c29a4cb8f5
Successfully built en-core-web-lg


In [1]:
!pip install bert-tensorflow

     |████████████████████████████████| 64 kB 2.0 MB/s 


In [4]:
#%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
!ln -s /content/gdrive/MyDrive/ /mydrive
!ls /mydrive

'2020 slides'
 2021-online-language-offline-impact-main
 An_Xception_Based_Convolutional_Neural_Network_for_Scene_Image_Classification_with_Transfer_Learning.pdf
'API key'
'Application for Lease- Form for Email (Applicant) (1).gdoc'
'Application for Lease- Form for Email (Applicant).gdoc'
'Application for Lease- Form for Email (Applicant).pdf'
'Colab Notebooks'
'CV -Xizhi Wu.pdf'
'FOW Documents'
 MyDrive
'NT15 LEASE 2021.gdoc'
'NT15 LEASE 2021.pdf'
'Review Group 9.gdoc'
 Xizhi_Wu_UnderGrad_Transcript.pdf
 教授资格证身份证.pdf


In [6]:
%cd gdrive/MyDrive/2021-online-language-offline-impact-main

/content/gdrive/MyDrive/2021-online-language-offline-impact-main


In [7]:
!ls

 a0529-a0611_UK		       Github
 a0529-a0611_US		       heatmap529_UK.ipynb
 csv			       heatmap530.ipynb
 csv.zip		       Heat_Map_Online_language.ipynb
 dataframe_csv		       __MACOSX
 dataframe_csv_EU	       mapping_UK
 dataframe_csv_UK	       mapping_US
 dataframe_result_529_UK.csv   non_violant1.csv
 dataframe_result_530_UK.csv   non_violant2.csv
 dataframe_result_531_UK.csv   UK
 dataframe_result_csv	       violant.csv
 dataframe_result_csv_EU      '新建 Microsoft PowerPoint 演示文稿.pptx'
 dataframe_result_csv_UK


In [8]:
import os
import collections
import ast
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime, timedelta
import spacy
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
from bert import tokenization
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import matplotlib.pyplot as plt

In [9]:
ct_path_601 = 'csv/US05_30.csv'

In [10]:
ct = pd.read_csv(ct_path_601)

In [11]:
ct.shape[1]

47

In [12]:
columns1=['platformId', 'platform', 'date', 'updated', 'type1', 'title', 'caption', 'description', 'message','expandedLinks', 'link', 'postUrl', 'subscriberCount', 'score', 'media_type', 'media_numl', 'actual_likeCount', 'actual_shareCount', 'actual_commentCount', 'actual_loveCount', 'actual_wowCount', 'actual_hahaCount', 'actual_sadCount', 'actual_angryCount', 'actual_thankfulCount', 'actual_careCount', 'expected_likeCount', 'expected_shareCount', 'expected_commentCount', 'expected_loveCount', 'expected_wowCount', 'expected_hahaCount', 'expected_sadCount', 'expected_angryCount', 'expected_thankfulCount', 'expected_careCount', 'account_id', 'account_name', 'account_handle', 'account_profileImage', 'account_accountType', 'account_pageAdminTopCountry', 'account_pageDescription', 'account_verified', 'account_pageCreatedDate', 'languageCode', 'legacyId', 'post_id']
columns2=['platformId', 'platform', 'date', 'updated', 'type1', 'title', 'caption', 'description', 'message', 'expandedLinks', 'link', 'postUrl', 'subscriberCount', 'score', 'media', 'actual_likeCount', 'actual_shareCount', 'actual_commentCount', 'actual_loveCount', 'actual_wowCount', 'actual_hahaCount', 'actual_sadCount', 'actual_angryCount', 'actual_thankfulCount', 'actual_careCount', 'expected_likeCount', 'expected_shareCount', 'expected_commentCount', 'expected_loveCount', 'expected_wowCount', 'expected_hahaCount', 'expected_sadCount', 'expected_angryCount', 'expected_thankfulCount', 'expected_careCount', 'account_id', 'account_name', 'account_handle', 'account_profileImage', 'account_accountType', 'account_pageAdminTopCountry', 'account_pageDescription', 'account_verified', 'account_pageCreatedDate', 'languageCode', 'legacyId', 'post_id']

In [13]:
#len(columns1)
len(columns2)

47

In [14]:
ct.columns=columns2

In [15]:
ct

,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id
0,1717735001821003_2576251915969303,Facebook,2020-05-30 22:01:36,2021-07-27 06:27:54,link,Spartanburg protesters show support for George...,newsbreak.com,The image of a black man held down and taking ...,Spartanburg protesters show support for George...,1.0,https://www.newsbreakapp.com/m/articles?nb_red...,https://www.facebook.com/1717735001821003/post...,1228,-0.875000,1.0,0,0,0,1,0,0,0,0,0,0,1,1,1,1,1,0,2,1,0,0,9975243,"Gaffney, SC Local - News Break",GaffneySCLocal,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Keep updated w/ the most interesting stories a...,False,2016-07-26 19:49:48,en,0,9975243|2576251915969303
1,122163047886_10157592533682887,Facebook,2020-05-30 22:01:35,2021-08-31 06:14:48,link,Military puts police units on alert for possib...,actionnewsnow.com,The US military on Friday ordered several poli...,The US military on Friday ordered several poli...,2.0,https://bit.ly/2XlTpFf,https://www.facebook.com/122163047886/posts/10...,134407,1.476190,1.0,41,10,79,3,6,0,5,11,0,0,19,13,21,6,6,7,22,11,0,3,25255,Action News Now,ActionNewsNow,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Your source for local news in the North State....,True,2009-09-01 18:48:05,en,0,25255|10157592533682887
2,1638944876418788_2524797221166878,Facebook,2020-05-30 22:01:34,2021-03-29 08:07:35,link,Demonstrators in Jacksonville join national ‘I...,newsbreak.com,"JACKSONVILLE, Fla. – People in Jacksonville jo...",Demonstrators in Jacksonville join national ‘I...,1.0,https://www.newsbreakapp.com/m/articles?nb_red...,https://www.facebook.com/1638944876418788/post...,0,-0.800000,1.0,1,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,1,12535316,"Jacksonville, FL Local - News Break",JacksonvilleFLLocal,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Keep updated w/ the most interesting stories a...,False,2016-07-20 18:38:30,en,0,12535316|2524797221166878
3,599127280248752_1531728960321908,Facebook,2020-05-30 22:01:34,2021-07-12 09:44:28,link,Comments From Chief Newlan On Current Events |...,newsbreak.com,Chief Newlan provides a brief comment related ...,Comments From Chief Newlan On Current Events,1.0,https://www.newsbreakapp.com/m/articles?nb_red...,https://www.facebook.com/599127280248752/posts...,2851,-22.000000,1.0,0,0,0,0,0,0,0,0,0,0,2,1,2,1,2,1,1,1,0,0,5970175,"Fort Myers, FL Local - News Break",FortMyersNews,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Keep updated w/ the most interesting stories a...,False,2016-07-19 19:11:22,en,0,5970175|1531728960321908
4,1670764379911054_2649530485367767,Facebook,2020-05-30 22:01:30,2021-05-22 23:16:40,link,Thousands Show Support for George Floyd in Far...,newsbreak.com,"FARGO, ND – A crowd in the thousands showed th...",Thousands Show Support for George Floyd in Far...,1.0,https://www.newsbreakapp.com/m/articles?nb_red...,https://www.facebook.com/1670764379911054/post...,1516,-22.000000,1.0,0,0,0,0,0,0,0,0,0,0,1,2,1,1,2,0,1,3,0,0,5917209,"Fargo, ND Local - News Break",FortMyersNews,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Keep updated w/ the most interesting stories a...,False,2016-08-03 23:57:15,en,0,5917209|2649530485367767
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [16]:
ct['message'] = ct['message'].fillna('')
ct['description'] = ct['description'].fillna('')
ct['title'] = ct['title'].fillna('')

In [17]:
nlp = spacy.load('en_core_web_lg', disable=["tok2vec", "tagger", "parser", "attribute_ruler", "lemmatizer"])

In [18]:
def ner(txt):
    ner = dict()

    doc = nlp(txt)
    for ent in doc.ents:
        if ent.label_ == 'GPE':
            if ent.text not in ner:
                ner[ent.text] = 1
            else:
                ner[ent.text] += 1
    return ner

In [19]:
def ner_inline(doc):
    ner = dict()

    for ent in doc.ents:
        if ent.label_ == 'GPE':
            if ent.text not in ner:
                ner[ent.text] = 1
            else:
                ner[ent.text] += 1
    return ner

In [20]:
def ner_pipe(texts):
    batch = []
    for doc in nlp.pipe(texts, batch_size=200, n_threads=16):
        batch.append(ner_inline(doc))
    return batch

In [21]:
ct['NER-msg'] = ner_pipe(ct['message'])

In [22]:
ct['NER-msg'] = ct['message'].apply(ner)

In [23]:
ct['NER-desc'] = ct['description'].apply(ner)

In [24]:
ct['NER-title'] = ct['title'].apply(ner)

In [25]:
ct[['NER-msg', 'NER-desc']].head()

,NER-msg,NER-desc
0,{'Spartanburg': 1},{'Spartanburg': 1}
1,"{'US': 1, 'Minnesota': 1}","{'US': 1, 'Minnesota': 1}"
2,{'Jacksonville': 1},"{'JACKSONVILLE': 1, 'Fla.': 1, 'Jacksonville':..."
3,{},{}
4,{},{'Fargo': 2}


In [26]:
ct.to_csv('a0530.csv')

In [27]:
ct=pd.read_csv('a0530.csv')

In [28]:
!ls

 a0529-a0611_UK		       dataframe_result_csv_UK
 a0529-a0611_US		       Github
 a0530.csv		       heatmap529_UK.ipynb
 csv			       heatmap530.ipynb
 csv.zip		       Heat_Map_Online_language.ipynb
 dataframe_csv		       __MACOSX
 dataframe_csv_EU	       mapping_UK
 dataframe_csv_UK	       mapping_US
 dataframe_result_529_UK.csv   non_violant1.csv
 dataframe_result_530_UK.csv   non_violant2.csv
 dataframe_result_531_UK.csv   UK
 dataframe_result_csv	       violant.csv
 dataframe_result_csv_EU      '新建 Microsoft PowerPoint 演示文稿.pptx'


In [29]:
###
#ct=pd.read_csv('a0601.csv')

In [30]:
ct

,Unnamed: 0,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id,NER-msg,NER-desc,NER-title
0,0,1717735001821003_2576251915969303,Facebook,2020-05-30 22:01:36,2021-07-27 06:27:54,link,Spartanburg protesters show support for George...,newsbreak.com,The image of a black man held down and taking ...,Spartanburg protesters show support for George...,1.0,https://www.newsbreakapp.com/m/articles?nb_red...,https://www.facebook.com/1717735001821003/post...,1228,-0.875000,1.0,0,0,0,1,0,0,0,0,0,0,1,1,1,1,1,0,2,1,0,0,9975243,"Gaffney, SC Local - News Break",GaffneySCLocal,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Keep updated w/ the most interesting stories a...,False,2016-07-26 19:49:48,en,0,9975243|2576251915969303,{'Spartanburg': 1},{'Spartanburg': 1},{'Spartanburg': 1}
1,1,122163047886_10157592533682887,Facebook,2020-05-30 22:01:35,2021-08-31 06:14:48,link,Military puts police units on alert for possib...,actionnewsnow.com,The US military on Friday ordered several poli...,The US military on Friday ordered several poli...,2.0,https://bit.ly/2XlTpFf,https://www.facebook.com/122163047886/posts/10...,134407,1.476190,1.0,41,10,79,3,6,0,5,11,0,0,19,13,21,6,6,7,22,11,0,3,25255,Action News Now,ActionNewsNow,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Your source for local news in the North State....,True,2009-09-01 18:48:05,en,0,25255|10157592533682887,"{'US': 1, 'Minnesota': 1}","{'US': 1, 'Minnesota': 1}",{'Minnesota': 1}
2,2,1638944876418788_2524797221166878,Facebook,2020-05-30 22:01:34,2021-03-29 08:07:35,link,Demonstrators in Jacksonville join national ‘I...,newsbreak.com,"JACKSONVILLE, Fla. – People in Jacksonville jo...",Demonstrators in Jacksonville join national ‘I...,1.0,https://www.newsbreakapp.com/m/articles?nb_red...,https://www.facebook.com/1638944876418788/post...,0,-0.800000,1.0,1,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,1,12535316,"Jacksonville, FL Local - News Break",JacksonvilleFLLocal,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Keep updated w/ the most interesting stories a...,False,2016-07-20 18:38:30,en,0,12535316|2524797221166878,{'Jacksonville': 1},"{'JACKSONVILLE': 1, 'Fla.': 1, 'Jacksonville':...",{'Jacksonville': 1}
3,3,599127280248752_1531728960321908,Facebook,2020-05-30 22:01:34,2021-07-12 09:44:28,link,Comments From Chief Newlan On Current Events |...,newsbreak.com,Chief Newlan provides a brief comment related ...,Comments From Chief Newlan On Current Events,1.0,https://www.newsbreakapp.com/m/articles?nb_red...,https://www.facebook.com/599127280248752/posts...,2851,-22.000000,1.0,0,0,0,0,0,0,0,0,0,0,2,1,2,1,2,1,1,1,0,0,5970175,"Fort Myers, FL Local - News Break",FortMyersNews,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Keep updated w/ the most interesting stories a...,False,2016-07-19 19:11:22,en,0,5970175|1531728960321908,{},{},{}
4,4,1670764379911054_2649530485367767,Facebook,2020-05-30 22:01:30,2021-05-22 23:16:40,link,Thousands Show Support for George Floyd in Far...,newsbreak.com,"FARGO, ND – A crowd in the thousands showed th...",Thousands Show Support for George Floyd in Far...,1.0,https://www.newsbreakapp.com/m/articles?nb_red...,https://www.facebook.com/1670764379911054/post...,1516,-22.000000,1.0,0,0,0,0,0,0,0,0,0,0,1,2,1,1,2,0,1,3,0,0,5917209,"Fargo, ND Local - News Break",Fo

In [31]:
ct['NER-msg'] = ct['NER-msg'].apply(ast.literal_eval)
ct['NER-desc'] = ct['NER-desc'].apply(ast.literal_eval)
ct['NER-title'] = ct['NER-title'].apply(ast.literal_eval)

In [32]:
#ct_ner = ct[((ct['NER-msg'] != {}) | (ct['NER-desc'] != {}) | (ct['NER-title'] != {}))]

In [33]:
###
ct_ner = ct

In [34]:
ct_ner[['NER-msg', 'NER-desc', 'NER-title']].head()

,NER-msg,NER-desc,NER-title
0,{'Spartanburg': 1},{'Spartanburg': 1},{'Spartanburg': 1}
1,"{'US': 1, 'Minnesota': 1}","{'US': 1, 'Minnesota': 1}",{'Minnesota': 1}
2,{'Jacksonville': 1},"{'JACKSONVILLE': 1, 'Fla.': 1, 'Jacksonville':...",{'Jacksonville': 1}
3,{},{},{}
4,{},{'Fargo': 2},{}


In [35]:
ct_ner['NER-msg'] = ct_ner['NER-msg'].apply(lambda x: list(set(x.keys())))
ct_ner['NER-desc'] = ct_ner['NER-desc'].apply(lambda x: list(set(x.keys())))
ct_ner['NER-title'] = ct_ner['NER-title'].apply(lambda x: list(set(x.keys())))

In [36]:
ct_ner['NER-msg']

0                       [Spartanburg]
1                     [Minnesota, US]
2                      [Jacksonville]
3                                  []
4                                  []
                     ...             
19648    [New York City, Minneapolis]
19649      [Los Angeles, Minneapolis]
19650                              []
19651                              []
19652                       [Oakland]
Name: NER-msg, Length: 19653, dtype: object

In [37]:
ct_ner['NER'] = ct_ner['NER-msg'] + ct_ner['NER-desc'] + ct_ner['NER-title']

In [38]:
ct_ner['NER']

0                  [Spartanburg, Spartanburg, Spartanburg]
1                [Minnesota, US, Minnesota, US, Minnesota]
2        [Jacksonville, Jacksonville, Minneapolis, JACK...
3                                                       []
4                                                  [Fargo]
                               ...                        
19648                         [New York City, Minneapolis]
19649                           [Los Angeles, Minneapolis]
19650                                            [Atlanta]
19651                      [Minneapolis, Denver, Colorado]
19652           [Oakland, San Jose, Minneapolis, San Jose]
Name: NER, Length: 19653, dtype: object

In [39]:
ct_ner['NER'] = ct_ner['NER'].apply(set)
ct_ner['NER'] = ct_ner['NER'].apply(list)

In [40]:
ct_ner['NER'] = ct_ner['NER'].apply(lambda x: [s.lower() for s in x])

In [41]:
ct_ner

,Unnamed: 0,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id,NER-msg,NER-desc,NER-title,NER
0,0,1717735001821003_2576251915969303,Facebook,2020-05-30 22:01:36,2021-07-27 06:27:54,link,Spartanburg protesters show support for George...,newsbreak.com,The image of a black man held down and taking ...,Spartanburg protesters show support for George...,1.0,https://www.newsbreakapp.com/m/articles?nb_red...,https://www.facebook.com/1717735001821003/post...,1228,-0.875000,1.0,0,0,0,1,0,0,0,0,0,0,1,1,1,1,1,0,2,1,0,0,9975243,"Gaffney, SC Local - News Break",GaffneySCLocal,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Keep updated w/ the most interesting stories a...,False,2016-07-26 19:49:48,en,0,9975243|2576251915969303,[Spartanburg],[Spartanburg],[Spartanburg],[spartanburg]
1,1,122163047886_10157592533682887,Facebook,2020-05-30 22:01:35,2021-08-31 06:14:48,link,Military puts police units on alert for possib...,actionnewsnow.com,The US military on Friday ordered several poli...,The US military on Friday ordered several poli...,2.0,https://bit.ly/2XlTpFf,https://www.facebook.com/122163047886/posts/10...,134407,1.476190,1.0,41,10,79,3,6,0,5,11,0,0,19,13,21,6,6,7,22,11,0,3,25255,Action News Now,ActionNewsNow,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Your source for local news in the North State....,True,2009-09-01 18:48:05,en,0,25255|10157592533682887,"[Minnesota, US]","[Minnesota, US]",[Minnesota],"[minnesota, us]"
2,2,1638944876418788_2524797221166878,Facebook,2020-05-30 22:01:34,2021-03-29 08:07:35,link,Demonstrators in Jacksonville join national ‘I...,newsbreak.com,"JACKSONVILLE, Fla. – People in Jacksonville jo...",Demonstrators in Jacksonville join national ‘I...,1.0,https://www.newsbreakapp.com/m/articles?nb_red...,https://www.facebook.com/1638944876418788/post...,0,-0.800000,1.0,1,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,1,12535316,"Jacksonville, FL Local - News Break",JacksonvilleFLLocal,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Keep updated w/ the most interesting stories a...,False,2016-07-20 18:38:30,en,0,12535316|2524797221166878,[Jacksonville],"[Jacksonville, Minneapolis, JACKSONVILLE, Fla.]",[Jacksonville],"[jacksonville, minneapolis, jacksonville, fla.]"
3,3,599127280248752_1531728960321908,Facebook,2020-05-30 22:01:34,2021-07-12 09:44:28,link,Comments From Chief Newlan On Current Events |...,newsbreak.com,Chief Newlan provides a brief comment related ...,Comments From Chief Newlan On Current Events,1.0,https://www.newsbreakapp.com/m/articles?nb_red...,https://www.facebook.com/599127280248752/posts...,2851,-22.000000,1.0,0,0,0,0,0,0,0,0,0,0,2,1,2,1,2,1,1,1,0,0,5970175,"Fort Myers, FL Local - News Break",FortMyersNews,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Keep updated w/ the most interesting stories a...,False,2016-07-19 19:11:22,en,0,5970175|1531728960321908,[],[],[],[]
4,4,1670764379911054_2649530485367767,Facebook,2020-05-30 22:01:30,2021-05-22 23:16:40,link,Thousands Show Support for George Floyd in Far...,newsbreak.com,"FARGO, ND – A crowd in the thousands showed th...",Thousands Show Support for George Floyd in Far...,1.0,https://www.newsbreakapp.com/m/articles?nb_red...,https://www.facebook.com/1670764379911054/post...,1516,-22.000000,1.0,0,0,0,0,0,0,0,0,0,0,1,2,1,1,2,0,1,3,0,0,5917

In [54]:
ct_ner['NER-1']=ct_ner['NER']

In [43]:
ct_ner['NER-1'][0]

['spartanburg']

In [44]:
!pip install -q geopy
!pip install -q pyproj

# basic utilities
import json
import requests
import pandas as pd
import pprint
import time

# reverse geocoding utilities
from geopy.geocoders import Nominatim

# geography projection utilities
import pyproj
from shapely.ops import transform
from shapely.geometry import Point
from functools import partial

     |████████████████████████████████| 6.3 MB 5.1 MB/s 


In [45]:
import os
import collections
import ast
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime, timedelta
import spacy
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
#from bert import tokenization
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import matplotlib.pyplot as plt

In [46]:
ct_ner

,Unnamed: 0,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id,NER-msg,NER-desc,NER-title,NER,NER-1
0,0,1717735001821003_2576251915969303,Facebook,2020-05-30 22:01:36,2021-07-27 06:27:54,link,Spartanburg protesters show support for George...,newsbreak.com,The image of a black man held down and taking ...,Spartanburg protesters show support for George...,1.0,https://www.newsbreakapp.com/m/articles?nb_red...,https://www.facebook.com/1717735001821003/post...,1228,-0.875000,1.0,0,0,0,1,0,0,0,0,0,0,1,1,1,1,1,0,2,1,0,0,9975243,"Gaffney, SC Local - News Break",GaffneySCLocal,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Keep updated w/ the most interesting stories a...,False,2016-07-26 19:49:48,en,0,9975243|2576251915969303,[Spartanburg],[Spartanburg],[Spartanburg],[spartanburg],[spartanburg]
1,1,122163047886_10157592533682887,Facebook,2020-05-30 22:01:35,2021-08-31 06:14:48,link,Military puts police units on alert for possib...,actionnewsnow.com,The US military on Friday ordered several poli...,The US military on Friday ordered several poli...,2.0,https://bit.ly/2XlTpFf,https://www.facebook.com/122163047886/posts/10...,134407,1.476190,1.0,41,10,79,3,6,0,5,11,0,0,19,13,21,6,6,7,22,11,0,3,25255,Action News Now,ActionNewsNow,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Your source for local news in the North State....,True,2009-09-01 18:48:05,en,0,25255|10157592533682887,"[Minnesota, US]","[Minnesota, US]",[Minnesota],"[minnesota, us]","[minnesota, us]"
2,2,1638944876418788_2524797221166878,Facebook,2020-05-30 22:01:34,2021-03-29 08:07:35,link,Demonstrators in Jacksonville join national ‘I...,newsbreak.com,"JACKSONVILLE, Fla. – People in Jacksonville jo...",Demonstrators in Jacksonville join national ‘I...,1.0,https://www.newsbreakapp.com/m/articles?nb_red...,https://www.facebook.com/1638944876418788/post...,0,-0.800000,1.0,1,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,1,12535316,"Jacksonville, FL Local - News Break",JacksonvilleFLLocal,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Keep updated w/ the most interesting stories a...,False,2016-07-20 18:38:30,en,0,12535316|2524797221166878,[Jacksonville],"[Jacksonville, Minneapolis, JACKSONVILLE, Fla.]",[Jacksonville],"[jacksonville, minneapolis, jacksonville, fla.]","[jacksonville, minneapolis, jacksonville, fla.]"
3,3,599127280248752_1531728960321908,Facebook,2020-05-30 22:01:34,2021-07-12 09:44:28,link,Comments From Chief Newlan On Current Events |...,newsbreak.com,Chief Newlan provides a brief comment related ...,Comments From Chief Newlan On Current Events,1.0,https://www.newsbreakapp.com/m/articles?nb_red...,https://www.facebook.com/599127280248752/posts...,2851,-22.000000,1.0,0,0,0,0,0,0,0,0,0,0,2,1,2,1,2,1,1,1,0,0,5970175,"Fort Myers, FL Local - News Break",FortMyersNews,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Keep updated w/ the most interesting stories a...,False,2016-07-19 19:11:22,en,0,5970175|1531728960321908,[],[],[],[],[]
4,4,1670764379911054_2649530485367767,Facebook,2020-05-30 22:01:30,2021-05-22 23:16:40,link,Thousands Show Support for George Floyd in Far...,newsbreak.com,"FARGO, ND – A crowd in the thousands showed th...",Thousands Show Support for George Floyd in Far...,1.0,https://www.newsbreakapp.com/m/articles?nb_red...,https://www.facebook.co

In [47]:
%cd mapping_US

/content/gdrive/My Drive/2021-online-language-offline-impact-main/mapping_US


In [48]:
!ls

dataframe0529_post.csv	    dataframe_mapping_0605.csv
dataframe_mapping_0529.csv  dataframe_mapping_0606.csv
dataframe_mapping_0530.csv  dataframe_mapping_0607.csv
dataframe_mapping_0531.csv  dataframe_mapping_0608.csv
dataframe_mapping_0601.csv  dataframe_mapping_0609.csv
dataframe_mapping_0602.csv  dataframe_mapping_0610.csv
dataframe_mapping_0603.csv  dataframe_mapping_0611.csv
dataframe_mapping_0604.csv


In [49]:
ct_mapping=pd.read_csv('dataframe_mapping_0530.csv',encoding='ISO-8859-1')

In [50]:
len(ct_mapping['City'])

1791

In [51]:
len(ct_ner['NER-1'])

19653

In [55]:
for k in range(len(ct_ner['NER-1'])):
  list=[]
  for i in ct_ner['NER-1'][k]:
    for j in range(len(ct_mapping['City'])):
      if(i==ct_mapping['City'][j]):
        list.append(ct_mapping['Mapping'][j])
  ct_ner['NER-1'][k]=list

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [56]:
ct_ner

,Unnamed: 0,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id,NER-msg,NER-desc,NER-title,NER,NER-1
0,0,1717735001821003_2576251915969303,Facebook,2020-05-30 22:01:36,2021-07-27 06:27:54,link,Spartanburg protesters show support for George...,newsbreak.com,The image of a black man held down and taking ...,Spartanburg protesters show support for George...,1.0,https://www.newsbreakapp.com/m/articles?nb_red...,https://www.facebook.com/1717735001821003/post...,1228,-0.875000,1.0,0,0,0,1,0,0,0,0,0,0,1,1,1,1,1,0,2,1,0,0,9975243,"Gaffney, SC Local - News Break",GaffneySCLocal,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Keep updated w/ the most interesting stories a...,False,2016-07-26 19:49:48,en,0,9975243|2576251915969303,[Spartanburg],[Spartanburg],[Spartanburg],[spartanburg],"[Spartanburg, Spartanburg County, South Caroli..."
1,1,122163047886_10157592533682887,Facebook,2020-05-30 22:01:35,2021-08-31 06:14:48,link,Military puts police units on alert for possib...,actionnewsnow.com,The US military on Friday ordered several poli...,The US military on Friday ordered several poli...,2.0,https://bit.ly/2XlTpFf,https://www.facebook.com/122163047886/posts/10...,134407,1.476190,1.0,41,10,79,3,6,0,5,11,0,0,19,13,21,6,6,7,22,11,0,3,25255,Action News Now,ActionNewsNow,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Your source for local news in the North State....,True,2009-09-01 18:48:05,en,0,25255|10157592533682887,"[Minnesota, US]","[Minnesota, US]",[Minnesota],"[minnesota, us]","[Minnesota, United States, United States]"
2,2,1638944876418788_2524797221166878,Facebook,2020-05-30 22:01:34,2021-03-29 08:07:35,link,Demonstrators in Jacksonville join national ‘I...,newsbreak.com,"JACKSONVILLE, Fla. – People in Jacksonville jo...",Demonstrators in Jacksonville join national ‘I...,1.0,https://www.newsbreakapp.com/m/articles?nb_red...,https://www.facebook.com/1638944876418788/post...,0,-0.800000,1.0,1,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,1,12535316,"Jacksonville, FL Local - News Break",JacksonvilleFLLocal,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Keep updated w/ the most interesting stories a...,False,2016-07-20 18:38:30,en,0,12535316|2524797221166878,[Jacksonville],"[Jacksonville, Minneapolis, JACKSONVILLE, Fla.]",[Jacksonville],"[jacksonville, minneapolis, jacksonville, fla.]","[Jacksonville, Duval County, Florida, United S..."
3,3,599127280248752_1531728960321908,Facebook,2020-05-30 22:01:34,2021-07-12 09:44:28,link,Comments From Chief Newlan On Current Events |...,newsbreak.com,Chief Newlan provides a brief comment related ...,Comments From Chief Newlan On Current Events,1.0,https://www.newsbreakapp.com/m/articles?nb_red...,https://www.facebook.com/599127280248752/posts...,2851,-22.000000,1.0,0,0,0,0,0,0,0,0,0,0,2,1,2,1,2,1,1,1,0,0,5970175,"Fort Myers, FL Local - News Break",FortMyersNews,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Keep updated w/ the most interesting stories a...,False,2016-07-19 19:11:22,en,0,5970175|1531728960321908,[],[],[],[],[]
4,4,1670764379911054_2649530485367767,Facebook,2020-05-30 22:01:30,2021-05-22 23:16:40,link,Thousands Show Support for George Floyd in Far...,newsbreak.com,"FARGO, ND – A crowd in the thousands showed th...",Thousands Show Support for George Floyd in Far...,1.0,https:/

In [57]:
ct_ner['NER-1'][12]

['Elmira, Chemung County, New York, 14904-1306, United States',
 'Minneapolis, Hennepin County, Minnesota, 55415:55454, United States']

In [58]:
ct_ner.to_csv('dataframe0530_post.csv')